# EZyRB

## Tutorial 1: Model Reduction on a vtk pressure field - offline phase

In this tutorial we will show the typical workflow for the construcion of the reduced basis. 
First of all we just import ezyrb package.

In [1]:
import ezyrb as ez

Here we define the parameters that we use to extract all the informations from the vtk file:
- output_name is the name of the variable (or output) we want to extract from the solution file;
- weights_name is the name of the weights to be extracted from the solution file for the computation of the errors;
- namefile_prefix is the path and prefix of the solution files. The files are supposed to be named with the same prefix, plus an increasing numeration (from 00) in the same order as the parameter points;
- file_format is the format of the solution files.

In [2]:
output_name = 'Pressure'
weights_name = 'Weights'
namefile_prefix = '../tests/test_datasets/matlab_0'
file_format = '.vtk'

Now we initialize the file_handler that deals with all the vtk files. Then we need to start it in order to extract the output on the corners of the parametric domain. 
You can view the pressure field (computed with the high-fidelity method) corresponding to the 4 corners of the parametric domain in the following pictures.

![](pictures/pressure_in_corners.png)

The code prints to the screen the leave-one-out error on the parametric points and suggests the new parameter value for the next simulation.

In [3]:
file_handler = ez.pod.Pod(output_name, weights_name, namefile_prefix, file_format)
file_handler.read_config('tests/test_datasets/setting.conf')
file_handler.initialize_snapshot()
file_handler.print_info()

Maximum error on the tassellation: 0.149130123954
New baricentric parameter value added to the triangulation [-0.29334384 -0.2312056 ]



Then the algorithm waits for a new sequential vtk output. To emphasise the iterative procedure we explicit the while True cycle. We think is the best way to interact with the package. If the user choose True he has to run a new simulation with the new parameter suggested before ([-0.29334384 -0.2312056]) and then place the new vtk output with the sequential name in the same folder as the others (namefile_prefix). If the user choose False the code writes out the structures needed for the online phase. Here we respond True and then False just as an example.

In [ ]:
while True:
    add = input('Add a new snapshot to the database? (True or False)  ')
    if add == True:
        file_handler.add_snapshot()
    else:
        file_handler.write_structures()
        break

For the sake of clarity, we show also the pressure field corresponding to the parametric point ([-0.29334384 -0.2312056])

![](pictures/field_04.png)